In [1]:
import numpy as np
import pandas as pd 
import nltk
import matplotlib.pyplot as plt
from wordcloud import WordCloud,STOPWORDS
from PIL import Image
from sklearn.utils import resample
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.probability import FreqDist
from transformers import RobertaTokenizer
from transformers import TFRobertaModel
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.layers import Flatten
from sklearn.metrics import precision_recall_curve

import tensorflow as tf
from sklearn.metrics import roc_auc_score, roc_curve,auc
from keras import regularizers
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix,f1_score,classification_report
from mlxtend.plotting import plot_confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import label_binarize
from sklearn.metrics import average_precision_score
from itertools import cycle

lemma = WordNetLemmatizer()
stopword = set(STOPWORDS)
nltk.download('omw-1.4')
%matplotlib inline
import nltk
nltk.download('stopwords')
import nltk
nltk.download('wordnet')


from nltk.corpus import brown
nltk.download("brown")
import warnings
warnings.filterwarnings("ignore")


[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\alexh\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\alexh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\alexh\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package brown to
[nltk_data]     C:\Users\alexh\AppData\Roaming\nltk_data...
[nltk_data]   Package brown is already up-to-date!


In [2]:
tf.config.get_visible_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

# Import Train Data

# Data preprocessing

In [3]:
%cd ..
from data_retriever import data1, data2

C:\Users\alexh\PycharmProjects\proj_bach
File simple_emotions.txt already exists. Not overwriting.
File simple_sentiments.txt already exists. Not overwriting.
File clean_tweet_emotions.txt already exists. Not overwriting.
File clean_tweet_sentiments.txt already exists. Not overwriting.
                                                Text Sentiment
0  During the period of falling in love, each tim...  Positive
1         When I was involved in a traffic accident.  Negative
2  When I was driving home after  several days of...  Negative
3   When I lost the person who meant the most to me.  Negative
4  The time I knocked a deer down - the sight of ...  Negative
Index(['Text', 'Sentiment'], dtype='object')
                                                   Text Sentiment
0     During the period of falling in love, each tim...  Positive
1            When I was involved in a traffic accident.  Negative
2     When I was driving home after  several days of...  Negative
3      When I lost the per

In [4]:
def clean_text(tweet):
    tweets = " ".join(filter(lambda x: x[0]!= '@' , tweet.split()))
    tweets = re.sub('[^a-zA-Z]', ' ', tweets)
    tweets = tweets.lower()
    tweets = tweets.split()
    tweets = [word for word in tweets if not word in set(stopwords.words('english'))]
    tweets = [lemma.lemmatize(word) for word in tweets]
    tweets = " ".join(tweets)
    return tweets

In [5]:
train_data = data2.astype(str).copy()

In [6]:
train_data['clean_tweet_text'] = train_data.Text.apply(clean_text)

In [7]:
print(train_data.head())

                                                Text Sentiment  \
0                                                nan   neutral   
1                     Sage Act upgrade list tommorow  Positive   
2  WAY HOMEGIRL BABY FUNERAL MAN HATE FUNERALS SH...  Negative   
3  eye  true hazel eyeand brilliant  Regular feat...  Positive   
4    ugh babe hugggzzz u  babe naamazed nga ako e...  Positive   

                                    clean_tweet_text  
0                                                nan  
1                     sage act upgrade list tommorow  
2  way homegirl baby funeral man hate funeral sho...  
3  eye true hazel eyeand brilliant regular featur...  
4  ugh babe hugggzzz u babe naamazed nga ako e ba...  


In [8]:
train_data=train_data.drop(["Text"],axis=1)

In [9]:
train_data.head()

,Sentiment,clean_tweet_text
0,neutral,nan
1,Positive,sage act upgrade list tommorow
2,Negative,way homegirl baby funeral man hate funeral sho...
3,Positive,eye true hazel eyeand brilliant regular featur...
4,Positive,ugh babe hugggzzz u babe naamazed nga ako e ba...


In [10]:
max_len=128
text_data=train_data["clean_tweet_text"]
label_data=train_data["Sentiment"]

# roberta_tokenizer

In [11]:
roberta_tokenizer = RobertaTokenizer.from_pretrained("roberta-base")

# roberta_model

In [12]:
import tensorflow as tf
print(tf.reduce_sum(tf.random.normal([1000, 1000])))

tf.Tensor(1044.412, shape=(), dtype=float32)


In [13]:
roberta_model = TFRobertaModel.from_pretrained("roberta-base")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'roberta.embeddings.position_ids', 'lm_head.bias']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infe

# Sample Text

In [14]:
text_data[1300]

'time elated vice said mr tyson'

In [15]:
encode_data=roberta_tokenizer.encode_plus(text_data[1300],add_special_tokens = True,max_length =40,pad_to_max_length = True,truncation=True)

In [16]:
encode_data

{'input_ids': [0, 958, 1615, 1070, 2626, 26, 475, 338, 326, 20216, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]}

In [17]:
test_input_id=np.asarray(encode_data['input_ids'])
test_attention_mask=np.asarray(encode_data['attention_mask'])


In [18]:
print(test_input_id.shape)
print(test_attention_mask.shape)

print(test_attention_mask.reshape(1,-1).shape)

(40,)
(40,)
(1, 40)


In [19]:
pip install transformers

In [20]:
import transformers

In [21]:
print(isinstance(roberta_model, transformers.TFRobertaModel))  # Should return True


True


In [22]:
output_data=roberta_model([test_input_id.reshape(1,-1),test_attention_mask.reshape(1,-1)])

type(output_data)

transformers.modeling_tf_outputs.TFBaseModelOutputWithPoolingAndCrossAttentions

In [23]:
output_data

TFBaseModelOutputWithPoolingAndCrossAttentions(last_hidden_state=<tf.Tensor: shape=(1, 40, 768), dtype=float32, numpy=
array([[[-0.06151576,  0.06928044, -0.02511747, ..., -0.11403868,
         -0.06508613, -0.01430185],
        [-0.22992018,  0.00537658, -0.2360584 , ..., -0.31573626,
         -0.1950437 ,  0.00210309],
        [ 0.07679282,  0.1645904 ,  0.03715213, ...,  0.05029377,
          0.02517287,  0.10758667],
        ...,
        [ 0.04703602, -0.03357879, -0.10649902, ..., -0.4143144 ,
         -0.0469461 ,  0.00187367],
        [ 0.04703602, -0.03357879, -0.10649902, ..., -0.4143144 ,
         -0.0469461 ,  0.00187367],
        [ 0.04703602, -0.03357879, -0.10649902, ..., -0.4143144 ,
         -0.0469461 ,  0.00187367]]], dtype=float32)>, pooler_output=<tf.Tensor: shape=(1, 768), dtype=float32, numpy=
array([[-2.50649244e-01,  9.93642956e-02,  1.59080058e-01,
         3.90634626e-01, -7.45110353e-03, -5.69602735e-02,
        -2.41493925e-01,  2.22275585e-01, -1.20153114e-

In [24]:
roberta_tokenizer.decode(encode_data["input_ids"])

'<s>time elated vice said mr tyson</s><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad><pad>'

In [25]:
final_text_data=train_data["clean_tweet_text"]
final_label_data=np.array(label_data)

In [26]:
new_final_label_data = np.array([0 if label=="Negative" else 1 if label=="neutral" else 2 for label in label_data])

In [27]:
new_final_label_data = np.array([0 if label=="Negative" else 1 for label in label_data])

In [28]:
print(new_final_label_data[:25])

[1 1 0 1 1 0 0 0 0 1 0 0 0 1 1 0 1 0 0 0 0 1 0 1 1]


In [29]:
print(final_text_data)

0                                                      nan
1                           sage act upgrade list tommorow
2        way homegirl baby funeral man hate funeral sho...
3        eye true hazel eyeand brilliant regular featur...
4        ugh babe hugggzzz u babe naamazed nga ako e ba...
                               ...                        
30725                tuesday woken oscar cornet practice x
30726           world didnt meso world definitely cnt away
30727                                     man robbed today
30728                                 youu jealousy losing
30729                                think baby dream time
Name: clean_tweet_text, Length: 30730, dtype: object


In [30]:
def roberta_encode(final_text_data,max_len):
    input_ids=[]
    attention_masks=[]
    
    for i in range(len(final_text_data)):
        encode_data=roberta_tokenizer.encode_plus(final_text_data[i],add_special_tokens=True,max_length=max_len,pad_to_max_length=True,return_attention_mask=True)
        input_ids.append(encode_data['input_ids'])
        attention_masks.append(encode_data["attention_mask"])
    return np.array(input_ids),np.array(attention_masks)
        

In [31]:
text_input_ids,text_attention_masks = roberta_encode(final_text_data,max_len)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [32]:
print('Text Input Ids Shape {} \nText Input Attention Mask Shape {} \nLabel Data shape {}'.format(text_input_ids.shape,text_attention_masks.shape,new_final_label_data.shape))

Text Input Ids Shape (30730, 128) 
Text Input Attention Mask Shape (30730, 128) 
Label Data shape (30730,)


In [33]:
X_train_input,X_test_input,Y_train_label,Y_test_label,train_mask,test_mask=train_test_split(text_input_ids,new_final_label_data,text_attention_masks,test_size=0.2,random_state=42,shuffle=True)

In [34]:
print('Train input shape {}\nTest input shape {}\nTrain label shape {}\nTest label shape {}\nTrain attention mask shape {}\nTest attention mask shape {}'.format(X_train_input.shape,X_test_input.shape,Y_train_label.shape,Y_test_label.shape,train_mask.shape,test_mask.shape))

Train input shape (24584, 128)
Test input shape (6146, 128)
Train label shape (24584,)
Test label shape (6146,)
Train attention mask shape (24584, 128)
Test attention mask shape (6146, 128)


In [35]:
print(Y_train_label[675:750])

[0 0 1 0 1 0 1 1 0 0 1 1 1 0 1 1 0 1 1 0 0 1 1 1 0 0 1 1 1 0 0 0 1 1 0 1 1
 0 1 0 1 0 1 1 1 0 0 1 1 1 0 1 0 1 0 1 1 1 0 0 1 0 0 1 0 0 1 1 1 1 0 0 0 0
 1]


# Create Roberta Model

In [36]:
from tensorflow.keras.layers import LeakyReLU


In [37]:
def Create_Roberta_Model():
    input_ids=tf.keras.Input(shape=(max_len,),dtype="int32")
    attention_masks=tf.keras.Input(shape=(max_len,),dtype="int32")
    
    roberta_model = TFRobertaModel.from_pretrained("roberta-base")
    output_dim = roberta_model(input_ids=input_ids, attention_mask=attention_masks)[0][:,0,:]
    
    dense_layer = tf.keras.layers.Dense(32, activation="relu")(output_dim)
    dropout = tf.keras.layers.Dropout(0.3)(dense_layer)
    final_layer = tf.keras.layers.Dense(2, activation='softmax')(dropout)

    model = tf.keras.models.Model(inputs=[input_ids, attention_masks], outputs=final_layer)
    
    return model


In [38]:
print(new_final_label_data)

[1 1 0 ... 0 0 0]


In [39]:
model=Create_Roberta_Model()

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)

optimizer = tf.keras.optimizers.Adam(lr=1e-5)

model.compile(loss=loss,optimizer=optimizer, metrics=["accuracy"])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaModel: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'roberta.embeddings.position_ids', 'lm_head.bias']
- This IS expected if you are initializing TFRobertaModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaModel were not initialized from the PyTorch model and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and infe

In [40]:
history=model.fit([X_train_input,train_mask],Y_train_label,batch_size=16,epochs=10,validation_data=([X_test_input,test_mask],Y_test_label), verbose=1)

Epoch 1/10
 633/1537 [===========>..................] - ETA: 3:16 - loss: 0.5190 - accuracy: 0.7488

KeyboardInterrupt: 

In [ ]:
def model_loss_and_accuracy(history):
    
    fig=plt.figure(figsize=(15,15))
    plt.subplot(221)
    plt.plot(history.history["accuracy"],marker="o",linestyle=":",markersize=10,color="m",label="accuracy")
    plt.plot(history.history["val_accuracy"],marker="D",linestyle=":",markersize=10,color="b",label="val_accuracy")
    plt.title("Model Accuracy\n",fontsize=20,color="darkorange")
    plt.xlabel("Number of Epochs",color="midnightblue",fontsize=15)
    plt.ylabel("Accuracy",color="midnightblue",fontsize=15)
    plt.grid(color = 'green', linestyle = '--', linewidth = 1)
    plt.legend(loc="best")
    plt.tight_layout()
    
  
    plt.subplot(222)
    plt.plot(history.history["accuracy"],marker="v",linestyle="-.",markersize=10,color="g",label="accuracy")
    plt.plot(history.history["val_accuracy"],marker="s",linestyle="--",markersize=10,color="r",label="val_accuracy")
    plt.title("Model Loss\n",fontsize=20,color="limegreen")
    plt.xlabel("Number of Epochs",color="midnightblue",fontsize=15)
    plt.ylabel("Loss",color="midnightblue",fontsize=15)
    plt.grid(color = 'green', linestyle = '--', linewidth = 1)
    plt.legend(loc="best")
    plt.tight_layout()
    plt.show()

In [ ]:
model_loss_and_accuracy(history)

# F1 Score

In [ ]:
def plot_f1_score(history):
    f1_scores = []
    for epoch in range(len(history.history['val_loss'])):
        model.set_weights(history.model.get_weights())
        predictions = model.predict([X_test_input, test_mask])
        predictions = [round(p[0]) for p in predictions]
    
        f1_scores.append(f1_score(Y_test_label, predictions, average='weighted'))
    plt.figure(figsize=(10,10))
    plt.plot(range(len(history.history['val_loss'])), f1_scores, label='F1 score',marker="*",color="g",linestyle="--",linewidth=4,markersize=8,markerfacecolor="r")
    plt.title("F1 Score\n",color="black",fontsize=20)
    plt.xlabel('Epochs',fontsize=15,color="black")
    plt.ylabel('F1 score',fontsize=15,color="black")
    plt.legend()
    plt.show()


plot_f1_score(history)


# Label Names

In [ ]:
label_name=["Positive","Racist and Sexist"]

In [ ]:
pred=model.predict([X_test_input, test_mask])
prediction=np.argmax(pred, axis=1)

# Accuracy Score

In [ ]:
print("Accuracy Score is",accuracy_score(Y_test_label,prediction))

# ROC AUC Score

In [ ]:
print("ROC AUC Score is {}".format(roc_auc_score(Y_test_label, pred[:,1])))

# Average Precision Score

In [ ]:
print(classification_report(Y_test_label, np.argmax(pred, axis=1)))